In [1]:
# %pip install torch
# %pip install torch_geometric

In [13]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np 
from torch_geometric.loader import DataLoader
import os
from torch_geometric.nn import Sequential, GCNConv,GATConv
import matplotlib.pyplot as plt
from torch.nn import Linear
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, accuracy_score
import time
import MyData as data
from matplotlib.lines import Line2D
from sklearn.linear_model import LinearRegression


from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
import GNN_model

In [14]:
hidden_size = 385
num_ly=2
sel_alg=0
num_to_reduce=64
view='xz'
folder_pkl='bigger_event_GCN_2'


In [15]:
%%time
data=data.dataset_preparation(root='/lustrehome/federicacuna/TB_Sept_2023_ml/Data/preprocessed/')
if view=='xz':
    fname='pi-_10GeV_4ly_1e-4'
else:
    fname='pi-_10GeV_4ly_1e-4_yz'
data_trk=data.get_more_file(0,101,fname)#952,fname)
train_dataset = data_trk
val_dataset =data.get_more_file(130,151,fname)
test_dataset =data.get(160,fname)
print('len_train_dataset ',len(train_dataset))
print('len_val_dataset ',len(val_dataset))
print('len_test_dataset ',len(test_dataset))

taking a list of file from  0  to  101
taking a list of file from  130  to  151
taking the  data_160.pt
len_train_dataset  1009998
len_val_dataset  210000
len_test_dataset  10000
CPU times: user 3min 40s, sys: 18.2 s, total: 3min 58s
Wall time: 3min 59s


In [16]:
%%time
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda
NVIDIA A100-PCIE-40GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.1 GB
CPU times: user 413 µs, sys: 338 µs, total: 751 µs
Wall time: 723 µs


In [17]:
%%time
NUM_FEATURES = train_dataset[0].x.shape[1]
print(NUM_FEATURES)
NUM_CLASSES =1
n_ly=4
model_list=['SageConv','SageConv2','GCN','GCN_2','GAT']

5
CPU times: user 102 µs, sys: 0 ns, total: 102 µs
Wall time: 106 µs


In [18]:
outname=f'best_model{model_list[sel_alg]}_ly{n_ly}_view_{view}.pkl'

In [20]:
if sel_alg>len(model_list):
    print('no algorithm available')
if sel_alg==0:
    model = GNN_model.SageConv(num_feat=NUM_FEATURES,num_class=NUM_CLASSES)
elif sel_alg==1:
    model = GNN_model.SageConv2(num_feat=NUM_FEATURES,num_class=NUM_CLASSES)
elif sel_alg==2:
    model = GNN_model.GCN(num_feat=NUM_FEATURES, num_class=NUM_CLASSES,num_to_reduce=64)
elif sel_alg==3:
    model = GNN_model.GCN_2(hidden_size=hidden_size, num_ly=num_ly, num_feat=NUM_FEATURES, num_class=NUM_CLASSES,num_to_reduce=64)
elif sel_alg==4:
    model = GNN_model.GAT(num_feat=NUM_FEATURES, num_class=NUM_CLASSES)

In [21]:
%%time
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
####criterion = torch.nn.CrossEntropyLoss()
criterion=torch.nn.functional.binary_cross_entropy#for sageconv

def train_model(train_data):
    model.to(device)
    model.train()
    for data in train_data:
        data.to(device)
        out = model(data) # Perform a single forward pass.
        labels=data.y.float().to(device)
        loss = criterion(out.squeeze(1).float(), labels)  # Compute the loss.
        
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    return model.to(device)

threshold=0.8
def evaluate_model(model,test_data):
    model.eval()
    correct = 0
    total_samples = 0
    model_outputs = []
    targets = []
    pred_class=[]

    for data in test_data:
        data.to(device)
        model_output = model(data)
        model_output.to(device)
        predicted_class = torch.where(model_output > threshold, 1, 0)
        
        correct += int((predicted_class == data.y.float()).sum())
        total_samples += len(data.y)

          # store these to get the loss
        model_outputs.extend(model_output.tolist())
        targets.extend(data.y.float().tolist())
        pred_class.extend(predicted_class.tolist())

    
    loss = criterion(torch.tensor(model_outputs).squeeze(1), torch.tensor(targets).float())
    accuracy = accuracy_score(targets, pred_class)
    precision = precision_score(targets, pred_class)
    recall = recall_score(targets, pred_class)
    
    return accuracy, loss, recall, precision 


CPU times: user 863 µs, sys: 1.83 ms, total: 2.69 ms
Wall time: 1.87 ms


In [22]:
test_batch_list=[]
start_value = 4032
increment = 64
iterations = 5
total_sum = 0

for i in range(iterations):
    current_value = start_value + (i* increment)
    
    test_batch_list.append(current_value)

In [23]:

test_batch_list.insert(0, 128)

In [24]:
test_batch_list

[128, 4032, 4096, 4160, 4224, 4288]

In [26]:
%%time
try:
    times = []
    train_accuracies = []
    validation_accuracies = []
    validation_losses = []
    train_losses = []
    val_recall=[]
    val_precision=[]
    train_recall=[]
    train_precision=[]
    
    for batch_train in range(len(test_batch_list)):
        start = time.time()
        print('batch ',batch_train, test_batch_list[batch_train])
        train_loader = DataLoader(train_dataset, batch_size=test_batch_list[batch_train],shuffle=True,pin_memory=True,num_workers=10,drop_last=True) 
        val_loader=DataLoader(val_dataset, batch_size=128,shuffle=True,pin_memory=True,drop_last=True) 
        # test_loader=DataLoader(test_dataset, batch_size=64,shuffle=True)

        train_accuracies.append([])
        validation_accuracies.append([])
        validation_losses.append([])
        train_losses.append([])
        val_recall.append([])
        val_precision.append([])
        train_recall.append([])
        train_precision.append([])   
        
        
        for epoch in range(1, 3):
            start_epoch = time.time()
            print('epoch ',epoch)
            model = train_model(train_loader)
            print(next(model.parameters()).is_cuda)

            train_acc, train_loss, train_rec, train_prec = evaluate_model(model,train_loader)
            
            train_losses[batch_train].append(train_loss)

            val_acc, val_loss, val_rec, val_prec = evaluate_model(model,val_loader)
            validation_losses[batch_train].append(val_loss)

            train_accuracies[batch_train].append(train_acc)
            validation_accuracies[batch_train].append(val_acc)

            train_precision[batch_train].append(train_prec)
            train_recall[batch_train].append(train_rec)

            val_precision[batch_train].append(val_prec)
            val_recall[batch_train].append(val_rec)

            # save the model if it is the better than any previous ones
            if val_loss.item() <= min(validation_losses[batch_train]).item():
                # torch.save(model, f'/lustrehome/federicacuna/TB_Sept_2023_ml/output_gnn_pkl/{folder_pkl}/{outname}')
                print(f'Epoch_stop: {epoch}')

            #torch.save(model, "best_model.pkl")
            if epoch % 1 == 0:
                print(f"Epoch {epoch}: Memoria GPU utilizzata: {torch.cuda.memory_allocated(device) / 1024**3:.2f} GB")
                print(f'Epoch: {epoch}, Train Acc: {train_acc:.4f}, Train Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}, Val Loss: {val_loss:.4f}, time :{time.time() - start_epoch}')
        print(time.time() - start)
        times.append(time.time() - start)
        

except RuntimeError as e:
    if "CUDA out of memory" in str(e):
        print("Errore: Memoria CUDA esaurita per batch ", batch_train)
    else:
        # Gestione di altri tipi di RuntimeError
        print("Errore:", e)  

batch  0 128
epoch  1
True
Epoch_stop: 1
Epoch 1: Memoria GPU utilizzata: 0.02 GB
Epoch: 1, Train Acc: 0.9559, Train Loss: 0.1270, Val Acc: 0.9574, Val Loss: 0.1227, time :191.1731460094452
epoch  2
True
Epoch_stop: 2
Epoch 2: Memoria GPU utilizzata: 0.02 GB
Epoch: 2, Train Acc: 0.9557, Train Loss: 0.1106, Val Acc: 0.9573, Val Loss: 0.1063, time :218.8551721572876
410.02870416641235
batch  1 4032
epoch  1
True
Epoch_stop: 1
Epoch 1: Memoria GPU utilizzata: 0.02 GB
Epoch: 1, Train Acc: 0.9619, Train Loss: 0.1004, Val Acc: 0.9635, Val Loss: 0.0963, time :96.37691235542297
epoch  2
True
Epoch_stop: 2
Epoch 2: Memoria GPU utilizzata: 0.02 GB
Epoch: 2, Train Acc: 0.9626, Train Loss: 0.0993, Val Acc: 0.9642, Val Loss: 0.0950, time :101.68007516860962
198.05716013908386
batch  2 4096
epoch  1
True
Epoch_stop: 1
Epoch 1: Memoria GPU utilizzata: 0.02 GB
Epoch: 1, Train Acc: 0.9629, Train Loss: 0.0985, Val Acc: 0.9644, Val Loss: 0.0946, time :104.92476224899292
epoch  2
True
Epoch_stop: 2
Epoch 

In [13]:
times

[415.2710831165314,
 373.8365707397461,
 406.7567677497864,
 381.89558839797974,
 375.7039797306061,
 412.0558304786682,
 404.0252447128296,
 394.52733612060547,
 420.3606700897217,
 434.0396246910095]

In [14]:
best_validation_loss = min(validation_losses)
best_epoch = validation_losses.index(best_validation_loss)
accuracy_at_best_epoch = validation_accuracies[best_epoch]
recall_at_best_epoch=val_recall[best_epoch]
precision_at_best_epoch=val_precision[best_epoch]
print(f"The best result was achieved after {best_epoch} epochs with a validation accuracy of {accuracy_at_best_epoch:.4f} and a loss of {best_validation_loss:.4f}")
print(f"The validation recall is {recall_at_best_epoch:.4f} and the precision is {precision_at_best_epoch:.4f}")

TypeError: unsupported format string passed to list.__format__

In [ ]:
fig, ax = plt.subplots()
#fig.set_size_inches(30.5, 15.5)

plt.plot(validation_losses,color='orange',label='validation loss')
plt.plot(train_losses,color='blue',label='training loss')

plt.axvline(x=best_epoch, color="green", linewidth=2, linestyle='dashed',label='best epoch')
#plt.legend(['Val Loss', 'Train Loss', "Best Epoch"])
plt.legend()


# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.title("Model training and validation loss per epoch", fontsize=10, fontweight='bold')
#plt.ylim(0.05,0.2)
# plt.savefig(f'/lustrehome/federicacuna/TB_Sept_2023_ml/output_gnn_img/{folder_pkl}/model_loss_{model_list[sel_alg]}_view_{view}.png')
plt.show()

In [ ]:
fig, ax = plt.subplots()
#fig.set_size_inches(30.5, 15.5)

plt.plot(validation_accuracies,color='orange',label='validation acc')
plt.plot(train_accuracies,color='blue',label='training acc')

plt.legend()

# Hide the right and top spines
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

plt.title("Model training and validation accuracy per epoch", fontsize=10, fontweight='bold')
# plt.ylim(0.9,1)
# plt.savefig(f'/lustrehome/federicacuna/TB_Sept_2023_ml/output_gnn_img/{folder_pkl}/model_acc_{model_list[sel_alg]}_view_{view}.png')
plt.show()